In [1]:
#Directions

#make sure you have installed cv2 with command: 
#  pip install opencv-python
#download the file called pose_iter_584000.caffemodel which is saved in this dropbox:
#  https://www.dropbox.com/s/3x0xambj2rkyrap/pose_iter_584000.caffemodel?dl=0
#download the file called pose_deploy.prototxt which is saved here:
#  https://github.com/CMU-Perceptual-Computing-Lab/openpose/tree/master/models/pose/body_25

#save both of these files in the same folder as this ipynb
#save a video in the same folder or subfolder


#issues:

# the model keeps recognizing exercise equipment in the background as body parts,
# i think using a different model(for multiple people simultaneously), 
# or adjusting confidence could help with this

# i could only make the scaling work by cropping the images to squares first
# not a big deal now with our training data, but it could cause problems in the future

# i think most of these .avi videos are 30 fps. if not, framestep() will need to be updated

# this script does not save the video, it only displays it in a popup window. 


In [79]:
import cv2

import csv

# Open the video file
video_path = 'DeepSquat1 (1).avi'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Cannot open camera")

# Define the keypoint mapping for this OpenPose body_25 model
keypoints_mapping = {
    0:  "Nose", 1:  "Neck", 2:  "RShoulder", 3:  "RElbow", 4:  "RWrist", 5:  "LShoulder", 6:  "LElbow",
    7:  "LWrist", 8:  "MidHip", 9:  "RHip", 10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee",
    14: "LAnkle", 15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe", 20: "LSmallToe",
    21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel"
}
#load the model
net = cv2.dnn.readNetFromCaffe('pose_deploy.prototxt', 'pose_iter_584000.caffemodel')

#Write frame information of joint locations into a csv file
f = open("DeepSquatJoint.csv", 'w', newline='')
w = csv.writer(f)
#for keypoint, label in keypoints_mapping:
    #w.write(str(keypoints_mapping.items())
#w.writerow(keypoints_mapping.values())
 
def framestep(cap):
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print("fps: %d" %(fps))
    print()
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    new_vid = []
    if (fps % 10 == 0): #we can easily do 10 fps
        stepsize = fps/1.2 # reset to 10
        print("total frames to be analyzed: ", frame_count / stepsize)
        curr_frame = 0
        i = 0
        while curr_frame < frame_count:
            ret, frame = cap.read()
            if curr_frame % stepsize == 0:
                print("starting frame: ", i)
                new_vid.append(pose(frame))
                i+=1
            curr_frame+=1
    
    else:
        print("NOT BUILT YET")
        pass
    
    return new_vid
    
def squarify(frame):
    height, width, _ = frame.shape
    min_dim = min(height, width)

    # Calculate the cropping dimensions
    crop_height = (height - min_dim) // 2
    crop_width = (width - min_dim) // 2

    # Crop the image equally from both sides to make it a square
    return frame[crop_height:crop_height+min_dim, crop_width:crop_width+min_dim] , min_dim

def pose(frame):
    
    frame, size = squarify(frame)
    blob = cv2.dnn.blobFromImage(frame, 1/255, (size, size),
                             (0, 0, 0), swapRB=False, crop=True)

    # run forward pass to get the pose estimation
    net.setInput(blob)
    output = net.forward()
    
    # Extract joint locations
    joint_locations = []
    csv_joint_locations = []

    for i in range(len(keypoints_mapping)): #-1 bc we dont want point for the background
        keypoint = output[0, i, :, :]
        min_val, confidence, min_loc, point = cv2.minMaxLoc(keypoint)

        #if confidence > 0.1:  # can adjust the confidence threshold if needed ???
        joint_locations.append((8 * int(point[0]), 8 * int(point[1]), 0)) #for testing/human readable
        csv_joint_locations.append(8 * int(point[0])) #to be printed into csv
        csv_joint_locations.append(8 * int(point[1])) #to be printed into csv
        csv_joint_locations.append(0) #to be printed into csv
        '''else:
            joint_locations.append(None) #for testing/human readable
            csv_joint_locations.append(8 * int(point[0])) #to be printed into csv
            csv_joint_locations.append(8 * int(point[1])) #to be printed into csv
        '''
    #joint_locations contains the locations of the detected joints and corresponding index

    '''for location in joint_locations:
        if location:
            x, y, index = location
            cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
            #cv2.putText(image, str(index), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    '''        
    #print joint locations into a csv file
    w.writerow(csv_joint_locations)
    
    return frame


#write the information + circles onto each frame and pop up a window for each frame
my_video = framestep(cap)
size = my_video[0].shape[1], my_video[0].shape[0]
print(size)
out = cv2.VideoWriter("real.avi", cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

'''for i in range(len(my_video)):
    out.write(my_video[i])
    cv2.imshow("video", my_video[i])
    cv2.waitKey(0)

cv2.destroyAllWindows()
'''
out.release()
cap.release()
f.close()

exit(1)

fps: 30

total frames to be analyzed:  20.68
starting frame:  0
starting frame:  1
starting frame:  2
starting frame:  3
starting frame:  4
starting frame:  5
starting frame:  6
starting frame:  7
starting frame:  8
starting frame:  9
starting frame:  10
starting frame:  11
starting frame:  12
starting frame:  13
starting frame:  14
starting frame:  15
starting frame:  16
starting frame:  17
starting frame:  18
starting frame:  19
starting frame:  20
(1080, 1080)


In [12]:
#to write the frames into a video format for viewer presentation

cap = cv2.VideoCapture(video_path)

def test():
    size = (1080, 1080)
    out = cv2.VideoWriter("deep.avi", cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
    for i in range(0, 200):
        f, s = squarify(cap.read()[1])
        s
        out.write(f)
    
    out.release()

test()
cap.release()

In [81]:
#split the movements in the file into 10 reps (240 frames/arrays)
#estimating by dividing by 10 for now to seperate the reps
#future me note: can divide each rep by the local minimum of the hip value within x number of frames to obtain the full repinfo

import numpy as np

def chunks(xs):
    # number of values to keep in each chunk, rounded down
    n = int(len(xs)/10) #nframes/10
    
    chunk = [] # list of 10 lists, first 9 with n values, 10th with all the rest
    for i in range(0, 9):
        chunk.append(xs[i * n:(i+1) * n])
        
    #chunk.append(xs[9 * n:len(xs)])
    chunk.append(xs[9 * n:10 * n])
    return chunk

#load file into a single array (all 240 arrays into 1)
f = open("DeepSquatJoint.csv", 'r', newline='')
w = csv.reader(f, delimiter = ',', quoting = csv.QUOTE_NONE)

raw_data = []

for row in w:
    raw_data.append(row)
    
split_data = chunks(raw_data)

print(len(split_data))
print(split_data)
f.close()


10
[[['504', '344', '504', '392', '464', '392', '424', '352', '768', '584', '552', '384', '616', '312', '656', '240', '504', '536', '472', '528', '456', '648', '400', '832', '536', '536', '552', '640', '400', '832', '488', '328', '512', '328', '480', '336', '528', '328', '', '', '', '', '400', '832', '', '', '912', '712', '400', '832'], ['504', '568', '504', '592', '448', '592', '376', '552', '336', '472', '552', '584', '632', '544', '672', '456', '504', '736', '464', '728', '392', '736', '400', '848', '536', '736', '600', '728', '608', '848', '488', '560', '512', '560', '472', '552', '528', '552', '656', '872', '656', '864', '600', '856', '360', '872', '360', '864', '408', '856']], [['504', '328', '504', '376', '456', '376', '384', '296', '360', '248', '552', '376', '624', '312', '664', '240', '504', '544', '472', '544', '440', '680', '400', '848', '536', '544', '560', '664', '608', '840', '496', '320', '520', '320', '480', '328', '536', '320', '656', '872', '656', '872', '600', '856'

In [119]:
#this step is applied after every subject has been analyzed by openpose and split by episode

combined = np.array(split_data * 10)
combined[combined == ''] = 0
combined = combined.astype(float)
print(combined.shape)
#print(combined)

#get the mean of every first value 
data_mean = np.mean(combined, axis = 0)
#Data_mean = repmat(mean(Correct_Xm,2), 1, size(Correct_Xm,2));

centered_data = combined - data_mean

# Scale the data between -1 and 1
scaling_value = np.ceil(max(np.max(centered_Correct_Data), abs(np.min(centered_Correct_Data))))
data_correct = centered_data/scaling_value;
print(data_correct)

(100, 2, 50)
[[[ 0.00101911 -0.0733758   0.         ... -0.18343949 -0.00917197
   -0.02547771]
  [ 0.          0.17121019  0.         ... -0.00917197  0.
    0.00101911]]

 [[ 0.00101911 -0.09375796  0.         ...  0.02038217  0.00101911
    0.00509554]
  [ 0.01019108  0.12025478  0.         ...  0.00101911  0.
    0.00101911]]

 [[ 0.00101911 -0.02242038  0.         ...  0.02038217  0.00101911
    0.00509554]
  [ 0.01019108 -0.0733758   0.01019108 ...  0.00101911  0.
    0.00101911]]

 ...

 [[ 0.00101911 -0.08356688  0.         ...  0.02038217  0.00101911
    0.00509554]
  [ 0.          0.03872611  0.         ...  0.00101911  0.
   -0.00917197]]

 [[ 0.00101911  0.06929936  0.         ...  0.02038217  0.00101911
   -0.00509554]
  [ 0.         -0.12433121  0.         ...  0.00101911  0.
    0.00101911]]

 [[-0.00917197 -0.10394904  0.         ...  0.02038217  0.00101911
    0.00509554]
  [ 0.         -0.13452229  0.         ...  0.00101911  0.
    0.00101911]]]
